In [1]:
import numpy as np
import dill as pickle

comp_ind_dict = {'water': 0,
                 'methane': 1,
                 'ethane': 2,
                 'propane': 3}

In [2]:
def get_stable_phases(x):
    index_get = list()
    color = 'black'
    if np.sum((x[4][0][:, 1] - x[4][0][:, 2])**2) < 1e-8:
        if (x[4][1][1] > x[4][1][2]) and (x[4][1][1] > 1e-10):
            index_get.append(2)
        elif (x[4][1][2] > 1e-10):
            index_get.append(2)
    else:
        if x[4][1][1] > 1e-10:
            index_get.append(1)
        if x[4][1][2] > 1e-10:
            index_get.append(2)
    for ii in [0, 3, 4]:
        if x[4][1][ii] > 1e-10:
            index_get.append(ii)
    if len(index_get) == 2:
        if sorted(index_get) == [0,1]:
            color = 'blue'
        elif sorted(index_get) == [0,2]:
            color = 'green'
        elif sorted(index_get) == [0,3]:
            color = 'pink'
        elif sorted(index_get) == [0,4]:
            color = 'orange'
        elif sorted(index_get) == [1,3]:
            color = 'teal'
        elif sorted(index_get) == [1,4]:
            color = 'gray'
        elif sorted(index_get) == [2,3]:
            color = 'olive'
        elif sorted(index_get) == [3,4]:
            color = 'brown'
        elif sorted(index_get) == [2,4]:
            color = 'purple'
        else:
            color = 'burgundy'
    return index_get, color

def make_ref_list(data):
    phase_stable_lookup = dict()
    type_ref_list = list()
    phase_type = 0
    for ii, x in enumerate(data):
        index_get, color = get_stable_phases(x)
        if sorted(index_get) in type_ref_list:
            phase_stable_lookup[type_ref_list.index(sorted(index_get))]['inds'].append(ii)
        else:
            phase_stable_lookup[phase_type] = {'phases': sorted(index_get), 'inds': [ii], 'color': color}
            phase_type += 1
            type_ref_list.append(sorted(index_get))
    return type_ref_list, phase_stable_lookup
        
def make_midpoint(xx, index):
    mean_pts = np.mean(xx[4][0][:, index], axis=1)
    if len(xx[2]) == 4:
        return mean_pts
    else:
        result = np.zeros([4])
        for ii in range(len(xx[2])):
            result[comp_ind_dict[xx[2][ii]]] = mean_pts[ii]
        return result
    
def make_full_mat(xx, index):
    data_mod = xx[4][0][:, index]
    if len(xx[2]) == 4:
        return data_mod
    else:
        result = np.zeros([4, len(index)])
        for ii in range(len(xx[2])):
            result[comp_ind_dict[xx[2][ii]], :] = data_mod[ii, :]
        return result
    
def prepare_3phase_data(data):
    phase_zone_dict = dict()
    ii = 0
    jj = 0
    ref_list, phase_stable_lookup = make_ref_list(data)
    while ii < len(ref_list):
        phase_inds = ref_list[ii]
        if len(phase_inds) == 3:
            data_subset = [data[i] for i in phase_stable_lookup[ii]['inds']]
            data_subsetfull = np.asarray([make_full_mat(x, phase_inds) for x in data_subset])
            reduce_cond = np.ones([len(data_subsetfull)], dtype=bool)
            
            data_3phase = np.unique(data_subsetfull[reduce_cond, :, :].round(6), axis=0)
            data_3phase = data_3phase[np.argsort(data_3phase[:, 1, 1]), :, :]
            data_mdpt = np.mean(data_3phase, axis=2)
            onecompfrac = data_mdpt[:, 1] / np.sum(data_mdpt[:, 1:], axis=1)
            midpt, midpt_ind = np.unique(onecompfrac.round(2), return_index=True)
            for midgroup in midpt_ind:
                data_group = data_3phase[midgroup, :, :]
                if len(data_group.shape) == 2:
                    data_group = data_group[np.newaxis, :, :]
                phase_zone_dict[jj] = {'inds': phase_inds,
                                       'data': data_group,
                                       'h2o_stats': {'max': np.squeeze(np.max(data_group, axis=-1))[0], 
                                                     'min': np.squeeze(np.min(data_group, axis=-1))[0]}}
                jj += 1
        ii += 1
    return phase_zone_dict

def prepare_2phase_data(data):
    phase_zone_dict = dict()
    ii = 0
    jj = 0
    ref_list, phase_stable_lookup = make_ref_list(data)
    while ii < len(ref_list):
        phase_inds = ref_list[ii]
        if len(phase_inds) == 2:
            data_subset = [data[i] for i in phase_stable_lookup[ii]['inds']]
            data_subsetfull = np.asarray([make_full_mat(x, phase_inds) for x in data_subset])
            data_2phase = np.unique(data_subsetfull.round(6), axis=0)
            data_2phase = data_2phase[np.argsort(np.unique(data_2phase, axis=2)[:, 1, 1]), :, :]
            phase_zone_dict[jj] = {'inds': phase_inds,
                                   'data': data_2phase,
                                   'h2o_stats': {'max': np.squeeze(np.max(data_2phase, axis=-1))[0], 
                                                 'min': np.squeeze(np.min(data_2phase, axis=-1))[0]}}
            jj += 1
        ii += 1
    return phase_zone_dict

def prepare_4phase_data(data):
    phase_zone_dict = dict()
    ii = 0
    jj = 0
    ref_list, phase_stable_lookup = make_ref_list(data)
    while ii < len(ref_list):
        phase_inds = ref_list[ii]
        if len(phase_inds) == 4:
            data_subset = [data[i] for i in phase_stable_lookup[ii]['inds']]
            data_subsetfull = np.asarray([make_full_mat(x, phase_inds) for x in data_subset])
            data_4phase = np.mean(data_subsetfull, axis=0)
            phase_zone_dict[jj] = {'inds': phase_inds,
                                   'data': data_4phase,
                                   'h2o_stats': {'max': np.squeeze(np.max(data_2phase, axis=-1))[0], 
                                                 'min': np.squeeze(np.min(data_2phase, axis=-1))[0]}}
            jj += 1
        ii += 1
    return phase_zone_dict

In [3]:
def gen_lookup_table(data, P, T):
    ref_list, phase_stable_lookup = make_ref_list(data)
    max_h2o = np.max([x[4][0][0,:] for x in data])
    min_h2o = np.min([x[4][0][0,:] for x in data])
    h2o_stats = {'max': max_h2o, 'min': min_h2o}
    phase_3_dict = prepare_3phase_data(data)
    phase_2_dict = prepare_2phase_data(data)
    phase_4_dict = prepare_4phase_data(data)

    for key_2p in phase_2_dict:
        inds_2p = phase_2_dict[key_2p]['inds']
        for key_3p in phase_3_dict:
            inds_3p = phase_3_dict[key_3p]['inds']
            overlap = set(inds_2p) & set(inds_3p)
            if len(overlap)==2:
                map_2pto3p = dict()
                for ii, ind_2p in enumerate(inds_2p):
                    for jj, ind_3p in enumerate(inds_3p):
                        if ind_2p == ind_3p:
                            map_2pto3p[ii] = jj
                addl_data = np.zeros((len(phase_3_dict[key_3p]['data']), 4, 2))
                addl_data[:, :, 0] = phase_3_dict[key_3p]['data'][:, :, map_2pto3p[0]]
                addl_data[:, :, 1] = phase_3_dict[key_3p]['data'][:, :, map_2pto3p[1]]
                phase_2_dict[key_2p]['data'] = np.vstack((phase_2_dict[key_2p]['data'], addl_data))

    full_data_dict = dict()
    full_data_dict['4_phase'] = phase_4_dict
    full_data_dict['3_phase'] = phase_3_dict
    full_data_dict['2_phase'] = phase_2_dict
    full_data_dict['h2o_stats'] = h2o_stats
    full_data_dict['PT_info'] = {'P': P, 'T': T}
    pickle.dump(full_data_dict, open('{0}bar_{1}C_data_dict.pkl'.format(P, T), 'w'))

In [4]:
# Template file: c1toc3_flashtable_110bar13.5C.pkl

import os
prefix = 'c1toc3_flashtable_110bar'
all_files = os.listdir(os.getcwd())
file_list = [f for f in all_files if prefix in f]
for filename in file_list:
    P = 110
    T = filename[len(prefix):-5]
    with open(filename, 'rb') as f:
        print(filename)
        gen_lookup_table(pickle.load(f), P, T)

c1toc3_flashtable_110bar15.5C.pkl
c1toc3_flashtable_110bar17.5C.pkl
c1toc3_flashtable_110bar13C.pkl
c1toc3_flashtable_110bar18C.pkl
c1toc3_flashtable_110bar14C.pkl
c1toc3_flashtable_110bar13.5C.pkl
c1toc3_flashtable_110bar15C.pkl
c1toc3_flashtable_110bar19C.pkl
c1toc3_flashtable_110bar18.5C.pkl
c1toc3_flashtable_110bar16C.pkl
c1toc3_flashtable_110bar14.5C.pkl
c1toc3_flashtable_110bar17C.pkl
c1toc3_flashtable_110bar16.5C.pkl


In [5]:
sample = pickle.load(open('110bar_13.5C_data_dict.pkl', 'rb'))

In [6]:
data_mdpt = np.mean(sample['3_phase'][0]['data'], axis=2)
onecompfrac = data_mdpt[:, 1] / np.sum(data_mdpt[:, 1:], axis=1)
t1, t2 = np.unique(onecompfrac.round(4), return_index=True)
#             reduce_cond = data_mdpt[:, 1] / np.sum(data_mdpt[:, 1:], axis=1)

In [7]:
t1, t2

(array([ 0.6048]), array([0]))